# Importing all the important libraries

In [1]:
import tensorflow as tf
import torch.nn as nn
import torch
import numpy as np
from tqdm import tqdm
from utils.dataframe import (
    save_model_variants_chunk_hf,
    load_model_variants_hf, save_model_variants_hf
)
from IPython.display import display
import tensorflow_hub as hub
import tensorflow_text as text
import fasttext
import fasttext.util
from laserembeddings import Laser
from utils.gpu import get_device

# Set Settings

In [2]:
tqdm.pandas()

# Common

In [3]:
# gpu device 
device = get_device()


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}
Using device: mps


# 2. Enhance Burmese Contextual Representations
- Use LASER, mUSE, and FastText for cross-lingual and morphology-aware training.
- Fine-tune mBERT, XLM-R on Burmese dataset after adding contextual embeddings.
- Train models again using combined embeddings.

## Data Preprocessing

### Split Train Data

In [ ]:
# chunk size
embedded_train_chunk_sizes = {
    "mbert": 20,
    "xlm-r": 20,
}
embedded_test_chunk_sizes = {
    "mbert": 5,
    "xlm-r": 5,
}

In [ ]:
# splti train data
for model_name in ["mbert", "xlm-r"]:
    for spt_name in ["bpe", "unigram"]:
        name = f"{model_name}_{spt_name}_train"
        train_dataset = load_model_variants_hf(name)

        # split and save
        save_model_variants_hf(train_dataset, f"embedded_{name}", embedded_train_chunk_sizes[model_name])

In [ ]:
# splti test data
for model_name in ["mbert", "xlm-r"]:
    for spt_name in ["bpe", "unigram"]:
        name = f"{model_name}_{spt_name}_test"
        train_dataset = load_model_variants_hf(name)

        # split and save
        save_model_variants_hf(train_dataset, f"embedded_{name}", embedded_test_chunk_sizes[model_name])

### Generate Embeddings

In [ ]:
# LASER Embeddings
laser = Laser()

In [ ]:
# mUSE (Multilingual Universal Sentence Encoder)
muse = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
# FastText for Morphology-Aware Training
fasttext.util.download_model('my', if_exists='ignore')  # Download Burmese FastText model
fasttext_model = fasttext.load_model('cc.my.300.bin')

In [ ]:
# Initialize projection layer (1836 → 768) in bf16
projection_layer = nn.Linear(1836, 768).to(torch.bfloat16)

In [ ]:
def generate_embeddings(batch):
    """
    Generates contextual embeddings using mUSE (TensorFlow), LASER, and FastText.
    Uses Hugging Face Datasets to process batches efficiently.
    Logs errors and stops execution on failure.
    """

    sentences = batch["target"]
    sentences = [str(text) if text is not None else "" for text in sentences]

    # Ensure all sentences are valid strings
    sentences = [s.replace("\n", " ").strip() if s else "" for s in sentences]

    # Generate mUSE embeddings (GPU-accelerated)
    batch_tensor = tf.convert_to_tensor(sentences)
    embeddings_muse = muse(batch_tensor).numpy()  # Shape: (batch_size, 512)

    # Generate FastText embeddings
    embeddings_fasttext = np.array([
        fasttext_model.get_sentence_vector(sentence) for sentence in sentences
    ])

    # Generate LASER embeddings
    embeddings_laser = laser.embed_sentences(sentences, lang="my")

    # Stack embeddings
    combined_embeddings = np.hstack([
        embeddings_muse,
        embeddings_laser,
        embeddings_fasttext
    ])

    # Convert to tensor & bf16
    tensor_embeddings = torch.tensor(combined_embeddings, dtype=torch.bfloat16)

    # Apply projection (1836 → 768)
    projected_embeddings = projection_layer(tensor_embeddings).detach().tolist()

    return {"contextual_embeddings": projected_embeddings}

In [ ]:
# model and spt
model_name = "xlm-r"
spt_name = "bpe"

#### Train

In [ ]:
# chunk num
chunk_num = 4

In [ ]:
# train path
chunk_dataset_path = f"embedded_{model_name}_{spt_name}_test"

In [ ]:
# load chunk dataset
chunk_dataset = load_model_variants_hf(chunk_dataset_path, chunk_num=chunk_num)
print(f"Chunk Dataset Length: {len(chunk_dataset)}")
display(chunk_dataset.to_pandas().head())

In [ ]:
# generate embeddings
chunk_dataset = chunk_dataset.map(generate_embeddings, batched=True, batch_size=1024)

In [ ]:
# display
display(chunk_dataset.to_pandas().head())

In [ ]:
# save chunk dataset
save_model_variants_chunk_hf(chunk_dataset, chunk_dataset_path, chunk_num=chunk_num)

In [ ]:
not_generated_list = []
for i in range(5):
    dataset = load_model_variants_hf(chunk_dataset_path, chunk_num=i)

    if "contextual_embeddings" not in dataset.to_pandas().columns:
        not_generated_list.append(i)

print(not_generated_list)